In [ ]:
#create base

In [ ]:
from mako.template import Template
from mako.runtime import Context
from io import StringIO
import random
import matplotlib.pyplot as plt
import random
import PySpice.Logging.Logging as Logging
from PySpice.Probe.Plot import plot
from MySpice import MySpice as spice
import numpy as np
from numpy import genfromtxt
import tensorflow as tf
N1=1000
N2=20000
nom=5
lab=3
ban= np.empty(N1)
noise=45

In [ ]:
def type_1(i):
    k=0    
    file_name = 'C:\\Users\\kipar\\Desktop\\Test\\CIR{}\\{}-{}.cir'.format(lab,k,i)
    mytemplate = Template(filename='C:\\Users\\kipar\\Desktop\\Test\\пример{}.txt'.format(k+1))
    buf = StringIO()
    ctx = Context(buf, r1=str(random.randint(10, 10000)))
    graf(ctx, buf, file_name, mytemplate, k, i)

In [ ]:
def type_2(i):
    k=1    
    file_name = 'C:\\Users\\kipar\\Desktop\\Test\\CIR{}\\{}-{}.cir'.format(lab,k,i)
    mytemplate = Template(filename='C:\\Users\\kipar\\Desktop\\Test\\пример{}.txt'.format(k+1))
    buf = StringIO()
    ctx = Context(buf, r1=str(random.randint(10, 10000)), c1=str(random.randint(100,100000)))
    graf(ctx, buf, file_name, mytemplate, k, i)

In [ ]:
def type_3(i):
    k=2  
    file_name = 'C:\\Users\\kipar\\Desktop\\Test\\CIR{}\\{}-{}.cir'.format(lab,k,i)
    mytemplate = Template(filename='C:\\Users\\kipar\\Desktop\\Test\\пример{}.txt'.format(k+1))
    buf = StringIO()
    ctx = Context(buf, r1=str(random.randint(10, 1000)), N1=str(random.uniform(1.,2.)))
    graf(ctx, buf, file_name, mytemplate, k, i)

In [ ]:
def type_4(i):
    k=3 
    file_name = 'C:\\Users\\kipar\\Desktop\\Test\\CIR{}\\{}-{}.cir'.format(lab,k,i)
    mytemplate = Template(filename='C:\\Users\\kipar\\Desktop\\Test\\пример{}.txt'.format(k+1))
    buf = StringIO()
    ctx = Context(buf, c1=str(random.randint(100, 5000)), N1=str(random.uniform(1.,2.)))
    graf(ctx, buf, file_name, mytemplate, k, i)

In [ ]:
def type_5(i):
    k=4
    file_name = 'C:\\Users\\kipar\\Desktop\\Test\\CIR{}\\{}-{}.cir'.format(lab,k,i)
    mytemplate = Template(filename='C:\\Users\\kipar\\Desktop\\Test\\пример{}.txt'.format(k+1))
    buf = StringIO()
    ctx = Context(buf, N1=str(random.uniform(1.,2.)), N2=str(random.uniform(1.,2.)))
    graf(ctx, buf, file_name, mytemplate, k, i)

In [ ]:
def graf(ctx, buf, file_name, mytemplate, k, i):
    mytemplate.render_context(ctx)
    with open(file_name,'w') as f:
        f.write(buf.getvalue())
    logger = Logging.setup_logging()
    circuit = spice.LoadFile('C:\\Users\\kipar\\Desktop\\Test\\CIR{}\\{}-{}.cir'.format(lab,k, i))
    input_data = spice.Init_Data(100, 5, 4700, noise)
    analysis = spice.CreateCVC1(circuit, input_data, 100, "input")
    spice.SaveFile(analysis,"C:\\Users\\kipar\\Desktop\\Test\\CSV{}\\{}.csv".format(lab,i))
    #png(i, analysis, k)

In [ ]:
def png(i, analysis, k):
    if (i%1==0):
            figure1 = plt.figure(1, (20, 10))
            plt.grid()
            plt.plot(analysis.input_dummy, analysis.VCurrent, color = 'black')
            plt.xlabel('Напряжение [В]')
            plt.ylabel('Сила тока [А]')
            plt.xlim (-5, 5)
            plt.axis('off')
            plt.show()
            figure1.savefig('C:\\Users\\kipar\\Desktop\\Test\\PNG{}\\{}-{}.png'.format(lab,k, i), transparent=True, facecolor = "w")

In [ ]:
def base():
    for i in range(0, N1):       
        if i<(N1/nom):
            type_1(i)
        if (i>=(N1/nom) and i<(N1*2/nom)):
            type_2(i)
        if (i>=(N1*2/nom) and i<((N1*3)/nom)):
            type_3(i)
        if (i>=(N1*(3/nom)) and i<N1*4/nom):
            type_4(i)
        if (i>=(N1*(4/nom)) and i<(N1*5/nom)):
            type_5(i)
        

In [ ]:
base()


In [ ]:
#creat dataset

In [ ]:
#сдвиг массива
def shift(a, n):
    shif=np.zeros((n, 2, 100))
    for i in range(0, n):    
        sdvig=np.random.randint(0,100)
        sdvig=-sdvig
        for j in range(0,2):
            for m in range(0,100):            
                if (m+sdvig<100):
                    shif[i, j, m]=a[i, j, m+sdvig]
                if (m+sdvig>=100):
                    shif[i, j, m]=a[i, j, m+sdvig-100]
                if m+sdvig<0:
                    shif[i, j, m]=a[i, j, m+sdvig+100]
    return(shif)

In [ ]:
#создание y1 y2
def y_1(t, n, k):
    y = np.empty(n, dtype=int)
    for i in range(0,n):
        if i<(n/k):
            y[i]=0
        if (i>=(n/k) and i<(n*2/k)):
            y[i]=1
        if (i>=(n*2/k) and i<(n*3/k)):
            y[i]=2
        if (i>=(n*3/k) and i<(n*4/k)):
            y[i]=3
        if (i>=(n*4/k) and i<(n*5/k)):
            y[i]=4
    return(y)     


In [ ]:
#создание y x
def dataset(n, l, k):
    t = np.random.permutation(n)
    y1=y_1(t,n, k)
    lam = np.empty((n, 2, 100))
    for i in range(0,n):
        lam[i, :, :]=genfromtxt('C:\\Users\\kipar\\Desktop\\Test\\CSV{}\\{}.csv'.format(l,i), delimiter=';')
    x=np.empty((n, 2, 100))
    y = np.empty(n, dtype=int)
    for i in range(0,n):
        for j in range(0,2):
            x[i, j, :]=lam[t[i], j, :]/max(abs(lam[t[i], j, :]))
        y[i]=y1[t[i]]
    return(shift(x, n), y)

In [ ]:
x_test, y_test = dataset(N1, lab, nom)

In [ ]:
x_train, y_train = dataset(1000, 3, 5)

In [ ]:
# learning NS

In [ ]:
model = tf.keras.models.Sequential([
tf.keras.layers.Flatten(input_shape=(2, 100)),
tf.keras.layers.Dense(100, activation=tf.nn.relu),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(5, activation=tf.nn.softmax)])
model.compile(optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])
history=model.fit(x_test, y_test, epochs=120, validation_split=0.5)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(x_train, y_train)

In [ ]:
print(x_test)

In [ ]:
predictions=model.predict(x_train)
for i in range(0, 500):
    print(np.argmax(predictions[i]), " ", y_train[i])
    if (np.argmax(predictions[i])!=y_train[i]):
        figure1 = plt.figure(1, (20, 10))
        plt.grid()
        plt.plot(x_train[i, 0, :], x_train[i, 1, :])
        plt.xlabel('Напряжение [В]')
        plt.ylabel('Сила тока [А]')
        plt.xlim (-5, 5)
        plt.show()